# 4. VGG16 FullModel (without any training)

https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

In [1]:
from keras.applications import vgg16
from keras.models import Model
import keras
import os,shutil,math,scipy,cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical
import pandas as pd
from keras.applications.vgg16 import VGG16,preprocess_input

Using TensorFlow backend.


In [2]:
def redo_kreas_category(arr):
    redo=[]
    for i in arr:
        redo.append(list(i).index(1))
    return redo

- Get the chosen categories

In [3]:
lists=pd.read_csv('categories.csv')
lists=list(lists['0'])
len(lists)

60

## First Chunk

- We have to do it in chunks, due to memory errors. Since we will only use the model for predictions, we will be fine

In [4]:
firstchunk = int(11386/2)
firstchunk

5693

In [5]:
true_labels = []
imgsize=224
#result_array = np.zeros((20580, imgsize, imgsize, 3))
#result_array = np.zeros((11386, imgsize, imgsize, 3))
result_array = np.zeros((firstchunk, imgsize, imgsize, 3))
cur_dir = os.path.dirname(os.path.realpath(os.getcwd()))
paths=cur_dir+'/Project/Images/'
# we have 20580 images


j=0

for i in lists:
    images=os.listdir(paths+i)
    label=i[10:]
    for k in images:
        if(j==5693):
            break
        img = cv2.imread(paths+i+'/'+k,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(imgsize,imgsize))
        true_labels.append(label)
        result_array[j,:,:,:]=np.array(img)
        j=j+1

In [8]:
result_array[-1]

array([[[ 95.,  82.,  51.],
        [130.,  89.,  55.],
        [255., 238., 192.],
        ...,
        [ 32.,  28.,  20.],
        [ 28.,  26.,  17.],
        [ 33.,  33.,  23.]],

       [[126., 111.,  80.],
        [135.,  93.,  60.],
        [252., 214., 170.],
        ...,
        [ 49.,  39.,  31.],
        [ 47.,  36.,  27.],
        [ 39.,  30.,  20.]],

       [[126., 109.,  78.],
        [161., 118.,  85.],
        [224., 183., 140.],
        ...,
        [ 69.,  51.,  40.],
        [ 70.,  49.,  39.],
        [ 70.,  50.,  39.]],

       ...,

       [[ 76.,  70.,  63.],
        [ 89.,  83.,  76.],
        [ 92.,  88.,  83.],
        ...,
        [ 54.,  56.,  50.],
        [ 62.,  65.,  55.],
        [ 55.,  58.,  48.]],

       [[ 77.,  71.,  64.],
        [ 93.,  87.,  80.],
        [ 92.,  88.,  83.],
        ...,
        [ 45.,  46.,  42.],
        [ 63.,  64.,  55.],
        [ 62.,  61.,  53.]],

       [[ 84.,  78.,  71.],
        [ 93.,  87.,  80.],
        [ 92.,  

In [9]:
result_array.shape

(5693, 224, 224, 3)

In [10]:
len(true_labels)

5693

In [11]:
len(np.unique(np.array(true_labels)))

28

- Preprocessing:
The image pixels need to be prepared in the same way as the ImageNet training data was prepared. Specificall from the paper: The only preprocessing we do is subtracting the mean RGB value, computed on the training set, from each pixel.

In [12]:
# prepare the image for the VGG model
image = preprocess_input(result_array)

- Get the model and predict it

In [13]:
# predict the probability across all output classes (in this case imagenet have 1000 classes, so 1000 classes)
# predict the probability across all output classes (in this case imagenet have 1000 classes, so 1000 classes)
model = VGG16()
yhat = model.predict(image)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
len(yhat)

5693

In [15]:
yhat[0]

array([4.92606123e-10, 2.78100543e-09, 2.04385686e-09, 9.99947236e-10,
       4.20686863e-09, 1.61428879e-08, 1.06027741e-07, 1.04442883e-07,
       1.21686503e-06, 1.24255228e-09, 7.71178399e-10, 1.62586622e-09,
       9.26604926e-09, 1.11909833e-08, 2.43003528e-09, 7.47674145e-10,
       1.56809910e-09, 2.05574835e-07, 7.75842413e-09, 1.37835940e-07,
       3.95795441e-09, 9.75535173e-08, 1.44276657e-09, 5.52995116e-09,
       1.00656097e-08, 7.39506290e-10, 1.18179702e-10, 8.73564665e-10,
       8.69417782e-09, 1.69466343e-08, 1.81866161e-10, 2.25086016e-09,
       7.54997065e-10, 3.39802817e-08, 3.34472787e-08, 1.81168280e-09,
       1.83284339e-08, 7.85041632e-09, 5.00157471e-10, 6.66420163e-09,
       4.03434564e-10, 1.34900158e-09, 9.18273679e-10, 7.78274112e-10,
       2.35802516e-10, 5.66256952e-09, 5.79766102e-09, 2.83474577e-10,
       4.86389329e-09, 7.26419480e-11, 1.06987896e-09, 5.64700342e-10,
       3.98809735e-10, 5.33634859e-10, 2.98889180e-09, 2.57318583e-10,
      

- Encode the results

In [16]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
# decoder will return class id, classname and class of the prediction. so you need to take the maximum of it.
label = decode_predictions(yhat)

pred_label=[]
for i in label:
    pred_label.append(i[0][1])

40960/35363 [==================================] - 0s 1us/step


- Evaluate the results

In [17]:
np.unique(np.array(true_labels))

array(['Afghan_hound', 'Airedale', 'Australian_terrier',
       'Bernese_mountain_dog', 'Blenheim_spaniel', 'EntleBucher',
       'Great_Pyrenees', 'Ibizan_hound', 'Irish_wolfhound',
       'Lakeland_terrier', 'Leonberg', 'Maltese_dog', 'Newfoundland',
       'Norwegian_elkhound', 'Pomeranian', 'Saluki', 'Samoyed',
       'Scottish_deerhound', 'Sealyham_terrier', 'Shih-Tzu',
       'Siberian_husky', 'Tibetan_terrier', 'basenji', 'beagle', 'cairn',
       'chow', 'papillon', 'pug'], dtype='<U20')

In [18]:
np.unique(np.array(pred_label))

array(['Afghan_hound', 'African_grey', 'Airedale',
       'American_Staffordshire_terrier', 'American_black_bear',
       'American_egret', 'Angora', 'Appenzeller', 'Arabian_camel',
       'Arctic_fox', 'Australian_terrier', 'Bedlington_terrier',
       'Bernese_mountain_dog', 'Blenheim_spaniel', 'Border_collie',
       'Border_terrier', 'Boston_bull', 'Bouvier_des_Flandres',
       'Brittany_spaniel', 'Cardigan', 'Chesapeake_Bay_retriever',
       'Chihuahua', 'Christmas_stocking', 'Dandie_Dinmont', 'Doberman',
       'Egyptian_cat', 'English_foxhound', 'English_setter',
       'English_springer', 'EntleBucher', 'Eskimo_dog',
       'European_gallinule', 'French_bulldog', 'German_shepherd',
       'German_short-haired_pointer', 'Gordon_setter', 'Great_Dane',
       'Great_Pyrenees', 'Greater_Swiss_Mountain_dog', 'Ibizan_hound',
       'Indian_elephant', 'Irish_setter', 'Irish_terrier',
       'Irish_water_spaniel', 'Irish_wolfhound', 'Italian_greyhound',
       'Japanese_spaniel', 'Ke

In [19]:
print(accuracy_score(true_labels, pred_label))

0.5120323203934657


- Save the result

In [20]:
hold_name=[]
j=0
for i in lists:
    images=os.listdir(paths+i)
    for k in images:
        if(j==5693):
            break
        hold_name.append(k)
        j=j+1

In [21]:
df=pd.DataFrame({'pic_id':hold_name,'actual_labels':true_labels,'predict':pred_label})
df.to_csv('VGG_fullmodel_partone.csv')

## Second Chunk

- Get the data

In [5]:
true_labels = []
imgsize=224
#result_array = np.zeros((20580, imgsize, imgsize, 3))
#result_array = np.zeros((11386, imgsize, imgsize, 3))
result_array = np.zeros((firstchunk, imgsize, imgsize, 3))
cur_dir = os.path.dirname(os.path.realpath(os.getcwd()))
paths=cur_dir+'/Project/Images/'
# we have 20580 images


j=0
a=0
for i in lists:
    images=os.listdir(paths+i)
    label=i[10:]
    for k in images:
        if(j>=5693):
            img = cv2.imread(paths+i+'/'+k,cv2.IMREAD_COLOR)
            img = cv2.resize(img,(imgsize,imgsize))
            true_labels.append(label)
            result_array[a,:,:,:]=np.array(img)
            a=a+1
        j=j+1

In [6]:
result_array[-1]

array([[[ 30.,  43.,  95.],
        [ 40.,  51., 108.],
        [ 40.,  50., 111.],
        ...,
        [ 23.,  31.,  46.],
        [ 25.,  30.,  37.],
        [ 32.,  35.,  40.]],

       [[ 39.,  52., 108.],
        [ 46.,  58., 116.],
        [ 49.,  60., 122.],
        ...,
        [ 22.,  31.,  45.],
        [ 25.,  30.,  37.],
        [ 32.,  35.,  39.]],

       [[ 45.,  58., 116.],
        [ 49.,  61., 121.],
        [ 53.,  64., 129.],
        ...,
        [ 21.,  29.,  42.],
        [ 24.,  30.,  35.],
        [ 32.,  35.,  39.]],

       ...,

       [[ 91., 120., 156.],
        [115., 143., 179.],
        [119., 147., 181.],
        ...,
        [ 45.,  65., 108.],
        [ 54.,  73., 108.],
        [ 85., 104., 138.]],

       [[103., 128., 157.],
        [128., 152., 182.],
        [108., 130., 161.],
        ...,
        [ 47.,  60.,  96.],
        [ 66.,  84., 115.],
        [111., 129., 160.]],

       [[158., 177., 197.],
        [180., 198., 220.],
        [165., 1

In [7]:
len(true_labels)

5693

In [8]:
len(np.unique(np.array(true_labels)))

33

- Preprocessing

In [9]:
# prepare the image for the VGG model
image = preprocess_input(result_array)

- Get the model and predict it

In [10]:
# predict the probability across all output classes (in this case imagenet have 1000 classes, so 1000 classes)
# predict the probability across all output classes (in this case imagenet have 1000 classes, so 1000 classes)
model = VGG16()
yhat = model.predict(image)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
len(yhat)

5693

- Encode the results

In [13]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
# decoder will return class id, classname and class of the prediction. so you need to take the maximum of it.
label = decode_predictions(yhat)

pred_label=[]
for i in label:
    pred_label.append(i[0][1])

In [14]:
print(accuracy_score(true_labels, pred_label))

0.4700509397505709


- Save the result

In [15]:
hold_name=[]
j=0
for i in lists:
    images=os.listdir(paths+i)
    for k in images:
        if(j==5693):
            break
        hold_name.append(k)
        j=j+1

In [16]:
df=pd.DataFrame({'pic_id':hold_name,'actual_labels':true_labels,'predict':pred_label})
df.to_csv('VGG_fullmodel_parttwo.csv')

## Overall Accuracy

In [17]:
part_one=pd.read_csv('VGG_fullmodel_partone.csv')
part_two=pd.read_csv('VGG_fullmodel_parttwo.csv')

In [18]:
combined=part_one.append(part_two) 

In [19]:
combined.head()

,Unnamed: 0,pic_id,actual_labels,predict
0,0,n02085936_4070.jpg,Maltese_dog,Maltese_dog
1,1,n02085936_4924.jpg,Maltese_dog,Maltese_dog
2,2,n02085936_25908.jpg,Maltese_dog,Maltese_dog
3,3,n02085936_7142.jpg,Maltese_dog,Maltese_dog
4,4,n02085936_10073.jpg,Maltese_dog,Maltese_dog


In [21]:
print(accuracy_score(combined['actual_labels'], combined['predict']))

0.4910416300720183
